In [ ]:
from utils import css_from_file
css_from_file('style/style.css')

In [ ]:
import pandas as pd
import numpy as np

from sklearn.cross_validation import KFold
from sklearn.metrics import log_loss

Read data from the file __data/boehringer/(train|test).csv__ from the data folder. 

The first column is a binary variable that you want to predict. The rest are numericals

In [ ]:
def load(path):
    df = pd.read_csv(path)
    if "Activity" not in df.columns:
        df["Activity"] = np.nan
    return df.drop("Activity",axis=1), df.Activity
    
X_tr, y_tr = load("data/boehringer/train.csv")
X_te, y_te = load("data/boehringer/test.csv")

print("training data shape", X_tr.shape)
print("testing data shape", X_te.shape)

In [ ]:
X_tr.head()

Exercise
---------------------

Using the starter code below try to improve the solution

1. What kind of models you can use?
2. Try changing model parameters to get the best cross validation error.
3. Use pipeline to transform features before modeling:
   - use some feature selection mechanism
   - use dimension reduction method (pca, svd, etc)
   
Tip: It is ok to loop over models and datasets like this.

```python
for data in [pipeline_1, pipeline_2, pipeline_3]:
    for model in [model_1, model_2, model_3]:
        # do stuff
```

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from cross_validation import cross_val_apply
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import LogisticRegression, BayesianRidge, SGDClassifier
from sklearn.pipeline import make_pipeline, make_union
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomTreesEmbedding
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier

clf = GridSearchCV(RandomForestClassifier(n_jobs=-1, n_estimators=100), 
                   param_grid={'max_depth':[5,10,15]})

%time oof_predictions = cross_val_apply(clf, X_tr, y_tr, decision_func="predict_proba")

err = log_loss(y_tr, oof_predictions)
print("Your error is", err)
if err > 0.5:
    print("You can still improve :)")

Examples of classifiers 

<div class='spoiler'>

clf1 = RandomForestClassifier(n_estimators=10,n_jobs=-1)
clf2 = make_pipeline(PCA(), LogisticRegression())

clf3 = make_pipeline(
   make_union(
       RandomTreesEmbedding(n_estimators=10), 
       LazyTransformer()
   ), 
   LogisticRegression()
)

for clf in [clf1,clf2,clf3]:
    clf.fit(x,y)
    
clf = make_pipeline(make_union(make_pipeline(RandomTreesEmbedding(n_estimators=20), StandardScaler(with_mean=False)), 
                               make_pipeline(StandardScaler(with_mean=False), VarianceThreshold(0.1))),
                    MLPClassifier((15,), alpha=15.0, verbose=True))

clf = BaggingClassifier(make_pipeline(
                        make_union(RandomTreesEmbedding(n_estimators=10), 
                                   LazyTransformer()),
                        StandardScaler(with_mean=False), 
                        VarianceThreshold(0.001),
                        MLPClassifier((25,), alpha=10.0, verbose=False)), 
                        max_samples=0.75,
                        max_features=0.75,
                        n_estimators=10)
</div>

In [ ]:
clf.fit(X_tr, y_tr)